#Part 2

## Libraries to import/install

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
import spacy
from spacy.language import Language
from spacy.pipeline import EntityRecognizer
spacy_nlp = spacy.load("en_core_web_sm")

In [51]:
!pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 40.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [58]:
import stanza
from stanza.server import CoreNLPClient
stanza.download('en')
stanza_nlp = stanza.Pipeline('en')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [59]:
import pandas as pd
import json
import matplotlib.pyplot as plt

In [22]:
from statistics import mean

##2.1 Named entity recognition

In [5]:
with open ("/content/drive/MyDrive/DataScience_Project/preproc_texts.json", "r") as f :
  texts = json.load(f)
texts

{'Female_mathematicians': ["Mirka Miller (née Koutova, 9 May 1949 – 2 January 2016) was a Czech-Australian mathematician and computer scientist interested in graph theory and data security. She was a professor of electrical engineering and computer science at the University of Newcastle.\n\n\n\nMiller was born on 9 May 1949 in Rumburk, then part of Czechoslovakia, as the oldest in a family of five children. After attempting to escape Czechoslovakia in 1968, stopped because of her companion's illness, she became a student at Charles University before successfully escaping in 1969 and becoming a refugee in Australia.\nMiller earned a bachelor's degree from the University of Sydney in 1976, both in mathematics and computer science,\nand as a student also played volleyball for the New South Wales team and then the Australia women's national volleyball team.\nShe married ornithologist Ben Miller, became a computer programmer working with the Sydney Morning Herald and for NSW Parks and Wildl

###Spacy

In [60]:
#Function that returns a dictionnary with a list of lists of couples (entity, label) per biography for each category (hence two keys)

def get_entities_spacy(texts) :

  keys = list(texts.keys())
  entities_dict = {keys[0] : [], keys[1] : []}
  for key in keys :
    for text in texts[key] :
      doc = spacy_nlp(text)
      entities = []
      for ent in doc.ents :
        entities.append((ent.text, ent.label_))
      entities_dict[key].append(entities)

  return entities_dict

In [61]:
entities_dict = get_entities_spacy(texts)

In [62]:
#Storing the dictionnary of entities using spacy

with open("/content/drive/MyDrive/DataScience_Project/spacy_entities.json", "w") as f:
  json.dump(entities_dict, f)

In [63]:
df_sp_entities = pd.DataFrame(entities_dict)
df_sp_entities

,Female_mathematicians,Contemporary_artists
0,"[(Mirka Miller, PERSON), (Koutova, ORG), (9 Ma...","[(Allan Kaprow, PERSON), (August 23, 1927, DAT..."
1,"[(Irmgard Flügge-Lotz, PERSON), (Lotz, PERSON)...","[(Slobodan ""Braco"" Dimitrijević, PERSON), (18 ..."
2,"[(Lai-Sang Lily Young, ORG), (Chinese, NORP), ...","[(González-Torres, PERSON), (Felix Gonzalez-To..."
3,"[(Holly Krieger, PERSON), (the University of C...","[(Tomasz Urbanowicz, PERSON), (1959, DATE), (W..."
4,"[(Lia Bronsard, PERSON), (March 1963, DATE), (...","[(Gilbert Prousch, PERSON), (Gilbert Proesch, ..."
...,...,...
95,"[(Julia Kempe, PERSON), (French, NORP), (Germa...","[(Keith Sonnier, PERSON), (July 31, 1941, DATE..."
96,"[(Rachel Blodgett Adams, PERSON), (October 13,...","[(Kiki Smith, PERSON), (January 18, 1954, DATE..."
97,"[(Elisa Maria Eugenia, PERSON), (6 September 1...","[(Hans Haacke, PERSON), (August 12, 1936, DATE..."
98,"[(Garcia Wilkerson, PERSON), (August 12, 1901 ...","[(Bruno Fonseca, PERSON), (1958, DATE), (May 3..."


In [64]:
#Counting the number of entities per text

df_sp_entities['ent_count_fm'] = df_sp_entities['Female_mathematicians'].apply(lambda l: len(l))
df_sp_entities['ent_count_ca'] = df_sp_entities['Contemporary_artists'].apply(lambda l: len(l))

neworder = ['Female_mathematicians', 'ent_count_fm', 'Contemporary_artists', 'ent_count_ca']
df_sp_entities = df_sp_entities.reindex(columns=neworder)

df_sp_entities

,Female_mathematicians,ent_count_fm,Contemporary_artists,ent_count_ca
0,"[(Mirka Miller, PERSON), (Koutova, ORG), (9 Ma...",85,"[(Allan Kaprow, PERSON), (August 23, 1927, DAT...",160
1,"[(Irmgard Flügge-Lotz, PERSON), (Lotz, PERSON)...",169,"[(Slobodan ""Braco"" Dimitrijević, PERSON), (18 ...",59
2,"[(Lai-Sang Lily Young, ORG), (Chinese, NORP), ...",141,"[(González-Torres, PERSON), (Felix Gonzalez-To...",779
3,"[(Holly Krieger, PERSON), (the University of C...",37,"[(Tomasz Urbanowicz, PERSON), (1959, DATE), (W...",317
4,"[(Lia Bronsard, PERSON), (March 1963, DATE), (...",63,"[(Gilbert Prousch, PERSON), (Gilbert Proesch, ...",234
...,...,...,...,...
95,"[(Julia Kempe, PERSON), (French, NORP), (Germa...",138,"[(Keith Sonnier, PERSON), (July 31, 1941, DATE...",55
96,"[(Rachel Blodgett Adams, PERSON), (October 13,...",121,"[(Kiki Smith, PERSON), (January 18, 1954, DATE...",353
97,"[(Elisa Maria Eugenia, PERSON), (6 September 1...",57,"[(Hans Haacke, PERSON), (August 12, 1936, DATE...",292
98,"[(Garcia Wilkerson, PERSON), (August 12, 1901 ...",70,"[(Bruno Fonseca, PERSON), (1958, DATE), (May 3...",70


In [65]:
#Getting basic statistics

ent_count_fm = df_sp_entities['ent_count_fm'].tolist()
ent_count_ca = df_sp_entities['ent_count_ca'].tolist()

print("""Female mathematicians:
NE count per text min: %s,
NE count per text max: %s,
NE count per text average: %s
""" %(min(ent_count_fm), max(ent_count_fm), mean(ent_count_fm)))

print("""Contemporary artists:
NE count per text min: %s,
NE count per text max: %s,
NE count per text average: %s
""" %(min(ent_count_ca), max(ent_count_ca), mean(ent_count_ca)))

Female mathematicians:
NE count per text min: 14,
NE count per text max: 667,
NE count per text average: 75.64

Contemporary artists:
NE count per text min: 9,
NE count per text max: 1511,
NE count per text average: 279.05



In [66]:
#Getting the word count per entity, for each category

keys = ["Female_mathematicians", "Contemporary_artists"]

all_entities = {keys[0] : [], keys[1] : []}
for key in keys :
  for l in df_sp_entities[key].tolist() :
    all_entities[key].extend(l)

df_sp_fm_entities = pd.DataFrame(all_entities[keys[0]], columns=['Entity', 'Label'])
df_sp_fm_entities['word_count'] = df_sp_fm_entities['Entity'].apply(lambda x: len(x.split()))

df_sp_ca_entities = pd.DataFrame(all_entities[keys[1]], columns=['Entity', 'Label'])
df_sp_ca_entities['word_count'] = df_sp_ca_entities['Entity'].apply(lambda x: len(x.split()))

In [67]:
df_sp_fm_entities

,Entity,Label,word_count
0,Mirka Miller,PERSON,2
1,Koutova,ORG,1
2,9 May 1949,DATE,3
3,2 January 2016,DATE,3
4,Czech,NORP,1
...,...,...,...
7559,2008,DATE,1
7560,Function Spaces,ORG,2
7561,Differential Operators and Nonlinear Analysis,ORG,5
7562,Springer Basel AG,ORG,3


In [68]:
df_sp_ca_entities

,Entity,Label,word_count
0,Allan Kaprow,PERSON,2
1,"August 23, 1927",DATE,3
2,"April 5, 2006",DATE,3
3,American,NORP,1
4,Happening,WORK_OF_ART,1
...,...,...,...
27900,MungBeing,ORG,1
27901,Jesse Richards,PERSON,2
27902,Turkish,NORP,1
27903,Bakiniz,PERSON,1


In [69]:
#Getting basic statistics

word_count_fm = df_sp_fm_entities['word_count'].tolist()
word_count_ca = df_sp_ca_entities['word_count'].tolist()

print("""Female mathematicians:
Word count per NE min: %s,
Word count per NE max: %s,
Word count per NE average: %s
""" %(min(word_count_fm), max(word_count_fm), mean(word_count_fm)))

print("""Contemporary artists:
Word count per NE min: %s,
Word count per NE max: %s,
Word count per NE average: %s
""" %(min(word_count_ca), max(word_count_ca), mean(word_count_ca)))

Female mathematicians:
Word count per NE min: 1,
Word count per NE max: 15,
Word count per NE average: 1.9247752511898466

Contemporary artists:
Word count per NE min: 1,
Word count per NE max: 15,
Word count per NE average: 1.822576599175775



###Stanza

In [70]:
#Function that returns a dictionnary with a list of lists of couples (entity, label) per biography for each category (hence two keys)

def get_entities_stanza(texts) :

  keys = list(texts.keys())
  entities_dict = {keys[0] : [], keys[1] : []}
  for key in keys :
    for text in texts[key] :
      doc = stanza_nlp(text)
      entities = []
      for ent in doc.ents :
        entities.append((ent.text, ent.type))
      entities_dict[key].append(entities)

  return entities_dict

In [71]:
entities_dict = get_entities_stanza(texts)

KeyboardInterrupt: 

In [ ]:
#Storing the dictionnary of entities using stanza

with open("/content/drive/MyDrive/DataScience_Project/stanza_entities.json", "w") as f:
  json.dump(entities_dict, f)

In [ ]:
df_st_entities = pd.DataFrame(entities_dict)
df_st_entities

In [ ]:
#Counting the number of entities per text

df_st_entities['ent_count_fm'] = df_st_entities['Female_mathematicians'].apply(lambda l: len(l))
df_st_entities['ent_count_ca'] = df_st_entities['Contemporary_artists'].apply(lambda l: len(l))

neworder = ['Female_mathematicians', 'ent_count_fm', 'Contemporary_artists', 'ent_count_ca']
df_st_entities = df_st_entities.reindex(columns=neworder)

df_st_entities

In [ ]:
#Getting basic statistics

ent_count_fm = df_st_entities['ent_count_fm'].tolist()
ent_count_ca = df_st_entities['ent_count_ca'].tolist()

print("""Female mathematicians:
NE count per text min: %s,
NE count per text max: %s,
NE count per text average: %s
""" %(min(ent_count_fm), max(ent_count_fm), mean(ent_count_fm)))

print("""Contemporary artists:
NE count per text min: %s,
NE count per text max: %s,
NE count per text average: %s
""" %(min(ent_count_ca), max(ent_count_ca), mean(ent_count_ca)))

In [ ]:
#Getting the word count per entity, for each category

keys = ["Female_mathematicians", "Contemporary_artists"]

all_entities = {keys[0] : [], keys[1] : []}
for key in keys :
  for l in df_st_entities[key].tolist() :
    all_entities[key].extend(l)

df_st_fm_entities = pd.DataFrame(all_entities[keys[0]], columns=['Entity', 'Label'])
df_st_fm_entities['word_count'] = df_st_fm_entities['Entity'].apply(lambda x: len(x.split()))

df_st_ca_entities = pd.DataFrame(all_entities[keys[1]], columns=['Entity', 'Label'])
df_st_ca_entities['word_count'] = df_st_ca_entities['Entity'].apply(lambda x: len(x.split()))

In [ ]:
df_st_fm_entities

In [ ]:
df_st_ca_entities

In [ ]:
#Getting basic statistics

word_count_fm = df_st_fm_entities['word_count'].tolist()
word_count_ca = df_st_ca_entities['word_count'].tolist()

print("""Female mathematicians:
Word count per NE min: %s,
Word count per NE max: %s,
Word count per NE average: %s
""" %(min(word_count_fm), max(word_count_fm), mean(word_count_fm)))

print("""Contemporary artists:
Word count per NE min: %s,
Word count per NE max: %s,
Word count per NE average: %s
""" %(min(word_count_ca), max(word_count_ca), mean(word_count_ca)))

###Packages comparison

For the NE count per text

In [ ]:
plt.boxplot([df_sp_entities['ent_count_fm'].tolist(), df_st_entities['ent_count_fm'].tolist()], labels=["Spacy", "Stanza"])
plt.title("Boxplot for the number of NEs per Female mathematicians text depending on the package used")
plt.xlabel("Package")
plt.ylabel("Number of NEs")
plt.show()

In [ ]:
plt.boxplot([df_sp_entities['ent_count_ca'].tolist(), df_st_entities['ent_count_ca'].tolist()], labels=["Spacy", "Stanza"])
plt.title("Boxplot for the number of NEs per Contemporary artists text depending on the package used")
plt.xlabel("Package")
plt.ylabel("Number of NEs")
plt.show()

For the word count per NE

In [ ]:
plt.boxplot([df_sp_fm_entities['word_count'].tolist(), df_st_fm_entities['word_count'].tolist()], labels=["Spacy", "Stanza"])
plt.title("Boxplot for the word count per NE depending on the package used (for Female mathematicians)")
plt.xlabel("Package")
plt.ylabel("Number of words")
plt.show()

In [ ]:
plt.boxplot([df_sp_ca_entities['word_count'].tolist(), df_st_ca_entities['word_count'].tolist()], labels=["Spacy", "Stanza"])
plt.title("Boxplot for the word count per NE depending on the package used (for Contemporary artists)")
plt.xlabel("Package")
plt.ylabel("Number of words")
plt.show()